## Get the prediction dataset

In [355]:
import pandas as pd
import numpy as np

In [356]:
ebitda = pd.read_csv("../clean_data/ebitda_complete.csv")
cogs = pd.read_csv("../clean_data/cogs_complete.csv")
revenue = pd.read_csv("../clean_data/revenue_complete.csv")
market = pd.read_csv("../input_data/market_data.csv")
final_df = pd.read_csv("../clean_data/final_df.csv", index_col=0)

In [357]:
prediction_df = final_df[final_df["Year"]>=2022]

In [358]:
prediction_df.head()

,Company,Device,Market,SubMarket,Year,Value,Spendings,DevSpends,MarketSpends,SubMarketSpends,Cogs,Cogs_1,Cogs_2,Revenue,Revenue_1,Revenue_2,Ebitda,Ebitda_1,Ebitda_2
4,ABB,Amplifier/Comparator,Industrial,Power & Energy,2022,10.040442,257.635,242620.053,1010453.110,73245.445,19712.000000,19407.000000,18123.0,29446.00,28945.00,26134.0,4477.00,4641.00,2668.0
5,ABB,Amplifier/Comparator,Industrial,Power & Energy,2023,10.927740,291.701,241892.093,1067623.316,83379.756,9879.678571,19712.000000,19407.0,32174.52,29446.00,28945.0,5807.68,4477.00,4641.0
6,ABB,Amplifier/Comparator,Industrial,Power & Energy,2024,11.659800,322.803,243992.729,1131507.508,100954.916,10122.197857,9879.678571,19712.0,32970.35,32174.52,29446.0,5943.65,5807.68,4477.0
11,ABB,Analog Application Specific IC,Industrial,Power & Energy,2022,29.153250,646.449,1969197.965,1010453.110,73245.445,19712.000000,19407.000000,18123.0,29446.00,28945.00,26134.0,4477.00,4641.00,2668.0
12,ABB,Analog Application Specific IC,Industrial,Power & Energy,2023,33.304200,736.031,1782361.286,1067623.316,83379.756,9879.678571,19712.000000,19407.0,32174.52,29446.00,28945.0,5807.68,4477.00,4641.0


In [359]:
prediction_df = prediction_df.rename(columns={"Value": "CompanySpend"})

In [360]:
prediction_df.replace({2022:2025, 2023:2026, 2024:2027}, inplace=True)

### Financials

In [361]:
ebitda = ebitda[ebitda["Year"]>=2023]
cogs = cogs[cogs["Year"]>=2023]
revenue = revenue[revenue["Year"]>=2023]

In [362]:
comp_map = pd.read_csv("../input_data/financials_map.csv", delimiter= ",", encoding='unicode_escape', index_col = 0)
comp_map

,Company Name from Spend Tracker,Company Tag from S&P Global
0,ABB,ABB Ltd (SWX:ABBN)
1,Abbott Laboratories,Abbott Laboratories (NYSE:ABT)
2,Acer,Acer Incorporated (TWSE:2353)
3,Adient,Adient plc (NYSE:ADNT)
4,Advantest,Advantest Corporation (TSE:6857)
...,...,...
263,Yaskawa Electric Corporation,YASKAWA Electric Corporation (TSE:6506)
264,Yokogawa Electric,Yokogawa Electric Corporation (TSE:6841)
265,Zebra Technologies,Zebra Technologies Corporation (NasdaqGS:ZBRA)
266,ZF Group,ZF Group Inc.


In [363]:
replacement_dict = dict(zip(comp_map["Company Tag from S&P Global"], comp_map["Company Name from Spend Tracker"]))
cogs.Company = cogs.Company.replace(replacement_dict)
ebitda.Company = ebitda.Company.replace(replacement_dict)
revenue.Company = revenue.Company.replace(replacement_dict)

In [364]:
prediction_df.drop("Cogs", axis = 1, inplace=True)
prediction_df=prediction_df.merge(cogs, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Value":"Cogs"}, axis=1)

In [365]:
cogs["Year"] = cogs["Year"]+1

In [366]:
prediction_df.drop("Cogs_1", axis = 1, inplace=True)
prediction_df=prediction_df.merge(cogs, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Value":"Cogs_1"}, axis=1)

In [367]:
cogs["Year"] = cogs["Year"]+1

In [368]:
prediction_df.drop("Cogs_2", axis = 1, inplace=True)
prediction_df=prediction_df.merge(cogs, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Value":"Cogs_2"}, axis=1)

In [369]:
prediction_df.drop("Ebitda", axis = 1, inplace=True)
prediction_df=prediction_df.merge(ebitda, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Ebitda":"ebitda"}, axis=1)

In [371]:
ebitda["Year"] = ebitda["Year"]+1

In [372]:
prediction_df.drop("Ebitda_1", axis = 1, inplace=True)
prediction_df=prediction_df.merge(ebitda, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Ebitda":"Ebitda_1"}, axis=1)

In [373]:
ebitda["Year"] = ebitda["Year"]+1

In [374]:
prediction_df.drop("Ebitda_2", axis = 1, inplace=True)
prediction_df=prediction_df.merge(ebitda, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Ebitda":"Ebitda_2"}, axis=1)

In [375]:
prediction_df.drop("Revenue", axis = 1, inplace=True)
prediction_df=prediction_df.merge(revenue, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Revenue":"revenue"}, axis=1)

In [376]:
revenue["Year"] = revenue["Year"]+1

In [377]:
prediction_df.drop("Revenue_1", axis = 1, inplace=True)
prediction_df=prediction_df.merge(revenue, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Revenue":"Revenue_1"}, axis=1)

In [378]:
revenue["Year"] = revenue["Year"]+1

In [379]:
prediction_df.drop("Revenue_2", axis = 1, inplace=True)
prediction_df=prediction_df.merge(revenue, how="left", on=["Company", "Year"])
prediction_df = prediction_df.rename({"Revenue":"Revenue_2"}, axis=1)

In [383]:
prediction_df = prediction_df.rename({"ebitda":"Ebitda", "revenue":"Revenue"}, axis=1)

### Market

In [385]:
market = market[market["Region"]=="Worldwide"]

In [386]:
mapping_dict = {('Automotive Electronics Categories', 'ADAS'): ('Automotive', 'Auto ADAS'),
    ('Automotive Electronics Categories', 'Body & Convenience'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Chassis & Safety'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Connectivity & Telematics'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Hybrid & Electric Drive Train'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Infotainment & Cluster'): ('Automotive', 'Auto Infotainment'),
    ('Automotive Electronics Categories', 'Other Automotive'): ('Automotive', 'Other Auto & Aftermarket'),
    ('Automotive Electronics Categories', 'Powertrain & Vehicle Dynamics'): ('Automotive', 'Auto Powertrain'),
    ('Computing & Data Storage Categories', 'Data Center Servers'): ('Computer Platforms', 'Data Center Servers'),
    ('Computing & Data Storage Categories', 'Desktop PCs'): ('Computer Platforms', 'Desktop PCs'),
    ('Computing & Data Storage Categories', 'Flash Storage Cards'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Computing & Data Storage Categories', 'Hard Disk Drives'): ('Computer Peripherals & Storage', 'HDD'),
    ('Computing & Data Storage Categories', 'Notebook PCs'): ('Computer Platforms', 'Notebook PCs'),
    ('Computing & Data Storage Categories', 'Other Computing'): ('Computer Platforms', 'Other Computer Products'),
    ('Computing & Data Storage Categories', 'Other Data Storage'): ('Computer Peripherals & Storage', 'Other Storage'),
    ('Computing & Data Storage Categories', 'Other Peripherals'): ('Computer Peripherals & Storage', 'Other Peripherals'),
    ('Computing & Data Storage Categories', 'Smart Cards'): ('Computer Peripherals & Storage', 'Smart Cards'),
    ('Computing & Data Storage Categories', 'Solid-State Drives'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Computing & Data Storage Categories', 'Tablet PCs'): ('Computer Platforms', 'Tablet PCs'),
    ('Computing & Data Storage Categories', 'USB Flash Drive'): ('Computer Peripherals & Storage', 'Flash Cards/Drives'),
    ('Consumer Electronics Categories', 'Fitness & Wellness Wearable Electronics'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'LCD TV'): ('Consumer', 'TV'),
    ('Consumer Electronics Categories', 'Major Home Appliances'): ('Consumer', 'Appliance'),
    ('Consumer Electronics Categories', 'OLED TV'): ('Consumer', 'TV'),
    ('Consumer Electronics Categories', 'Other Audio/Video'): ('Consumer', 'Audio'),
    ('Consumer Electronics Categories', 'Other Consumer Electronics'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'Set-Top Boxes'): ('Consumer', 'STB'),
    ('Consumer Electronics Categories', 'Smart Speakers & Digital Assistants'): ('Consumer', 'Connected Consumer'),
    ('Consumer Electronics Categories', 'Smart Watches'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'VR Headsets'): ('Consumer', 'Other Consumer'),
    ('Consumer Electronics Categories', 'Video Game Consoles'): ('Consumer', 'Video Games'),
    ('Industrial Electronics Categories', 'Automation'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Building & Home Control'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Lighting'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Medical Electronics'): ('Industrial', 'Medical'),
    ('Industrial Electronics Categories', 'Military & Civil Aerospace'): ('Industrial', 'Military/Aerospace'),
    ('Industrial Electronics Categories', 'Other Industrial'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Power & Energy'): ('Industrial', 'Power & Energy'),
    ('Industrial Electronics Categories', 'Security & Video Surveillance'): ('Industrial', 'Other Industrial'),
    ('Industrial Electronics Categories', 'Test & Measurement'): ('Industrial', 'Test & Measurement'),
    ('Wired Communications Categories', 'Broadcast & Streaming Video'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Cable Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Cable CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Carrier Ethernet Switches & Routers'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'DSL Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'DSL CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Data Center Network Switches'): ('Wired Communications', 'DC Network & Threat Mitigation'),
    ('Wired Communications Categories', 'Enterprise Ethernet Switches & Routers'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'Enterprise UC & Voice'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'FTTH Aggregation Equipment'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'FTTH CPE'): ('Wired Communications', 'Carrier'),
    ('Wired Communications Categories', 'Low-Tier Consumer/SOHO Routers'): ('Wired Communications', 'Enterprise/SOHO'),
    ('Wired Communications Categories', 'Optical Equipment'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Other Wired Communications'): ('Wired Communications', 'Other Wired'),
    ('Wired Communications Categories', 'Threat Mitigation Products'): ('Wired Communications', 'DC Network & Threat Mitigation'),
    ('Wireless Communications Categories', 'Gray Market Handsets'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'High-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Low-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'M2M Modules'): ('Wireless Communications', 'Other Wireless'),
    ('Wireless Communications Categories', 'Media Tablets'): ('Wireless Communications', 'Media Tablets'),
    ('Wireless Communications Categories', 'Mid-Tier Smartphone'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Mobile Comm Infrastructure'): ('Wireless Communications', 'Infrastructure'),
    ('Wireless Communications Categories', 'Mobile Phone (ULCH, Entry, Feature)'): ('Wireless Communications', 'Handset'),
    ('Wireless Communications Categories', 'Other Wireless Communications'): ('Wireless Communications', 'Other Wireless'),
    ('Wireless Communications Categories', 'Wireless LAN Equipment'): ('Wireless Communications', 'Infrastructure')}

In [387]:
#Nb of keys
print(len(mapping_dict.keys()))

#Rename the market and submarket

def remapping(market, submarket):
    key = (market, submarket)
    return mapping_dict[key][0], mapping_dict[key][1]

market.Market, market.Submarket = zip(*market.apply(lambda x : remapping(x.Market, x.Submarket), axis = 1))
market.rename(columns = {"Submarket" : "SubMarket"}, inplace = True)
market.drop("Region", axis=1, inplace = True)

65


In [391]:
prediction_df = prediction_df.drop("Spendings", axis = 1)
prediction_df=prediction_df.merge(market, how="left", on=["Market", "SubMarket", "Device", "Year"])
prediction_df = prediction_df.rename({"Market size":"Spendings"}, axis=1)

In [395]:
prediction_df.drop(["DevSpends", "MarketSpends", "SubMarketSpends"], axis=1, inplace=True)

In [397]:
spend_bydevice = prediction_df[["Device", "Year", "Spendings"]].groupby(["Device", "Year"]).sum().reset_index().rename(columns = {"Spendings" : "DevSpends"})
spend_bymarket= prediction_df[["Market",  "Year","Spendings"]].groupby(["Market", "Year"]).sum().reset_index().rename(columns = {"Spendings" : "MarketSpends"})
spend_bysubmarket = prediction_df[["SubMarket", "Year", "Spendings"]].groupby(["SubMarket", "Year"]).sum().reset_index().rename(columns = {"Spendings" : "SubMarketSpends"})

In [398]:
prediction_df = prediction_df.merge(spend_bydevice, how = 'left', on = ["Device", "Year"])
prediction_df = prediction_df.merge(spend_bymarket, how = 'left', on = ["Market", "Year"])
prediction_df = prediction_df.merge(spend_bysubmarket, how = 'left', on = ["SubMarket", "Year"])

,Company,Device,Market,SubMarket,Year,CompanySpend,Cogs,Cogs_1,Cogs_2,Ebitda,Ebitda_1,Ebitda_2,Revenue,Revenue_1,Revenue_2,Spendings,DevSpends,MarketSpends,SubMarketSpends
0,ABB,Amplifier/Comparator,Industrial,Power & Energy,2025,10.040442,10364.717143,10122.197857,9879.678571,6271.240000,5943.650000,5807.68,34510.780000,32970.350000,32174.52,349.156,191535.122,888183.793,86272.406
1,ABB,Amplifier/Comparator,Industrial,Power & Energy,2026,10.927740,10607.236429,10364.717143,10122.197857,6997.000000,6271.240000,5943.65,36835.000000,34510.780000,32970.35,387.420,199490.357,943057.724,97851.512
2,ABB,Amplifier/Comparator,Industrial,Power & Energy,2027,11.659800,10849.755714,10607.236429,10364.717143,7210.000000,6997.000000,6271.24,38700.250000,36835.000000,34510.78,435.299,209122.124,1011041.131,108547.164
3,ABB,Analog Application Specific IC,Industrial,Power & Energy,2025,29.153250,10364.717143,10122.197857,9879.678571,6271.240000,5943.650000,5807.68,34510.780000,32970.350000,32174.52,1032.715,1432515.513,888183.793,86272.406
4,ABB,Analog Application Specific IC,Industrial,Power & Energy,2026,33.304200,10607.236429,10364.717143,10122.197857,6997.000000,6271.240000,5943.65,36835.000000,34510.780000,32970.35,1191.535,1472410.489,943057.724,97851.512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23919,iRobot,Interface,Consumer,Appliance,2026,8.350700,70910.041071,67334.039286,63758.037500,-23414.371429,-92.790000,-104.38,86546.660714,1177.430000,1076.24,59.001,183034.374,206901.104,16929.081
23920,iRobot,Interface,Consumer,Appliance,2027,5.781400,74486.042857,70910.041071,67334.039286,-28093.885714,-23414.371429,-92.79,91136.100000,86546.660714,1177.43,58.000,194332.053,203802.108,16902.000
23921,iRobot,Voltage Regulator/Reference,Consumer,Appliance,2025,23.717500,67334.039286,63758.037500,60182.035714,-92.790000,-104.380000,-175.67,1177.430000,1076.240000,990.27,432.000,521225.559,207314.255,17361.027
23922,iRobot,Voltage Regulator/Reference,Consumer,Appliance,2026,23.067700,70910.041071,67334.039286,63758.037500,-23414.371429,-92.790000,-104.38,86546.660714,1177.430000,1076.24,439.001,545117.123,206901.104,16929.081


In [ ]:
prediction_df.to_csv("../clean_data/prediction_df.csv")